In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf

In [12]:
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit


In [14]:
from sklearn.metrics import accuracy_score, make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score

In [15]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [16]:
news_df = pd.read_csv(r"C:\Users\pcc\Downloads archive (3)",sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\pcc\\Downloads archive (3)'

In [17]:
news_df.drop('filename',axis=1,inplace=True)
news_df.head()

NameError: name 'news_df' is not defined

# Preprocessing First Step

In [ ]:
news_df['category_id'] = news_df.category.factorize()[0]

In [ ]:
category_id_df = news_df[['category','category_id']].drop_duplicates().sort_values('category_id')


In [ ]:
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id','category']].values)

# Visualization

In [ ]:
pd.DataFrame(news_df.category.value_counts()).rename(columns={'category':'Count'}).style.bar(color='#137a63')

In [ ]:
news_df.drop_duplicates(subset=['category','title'],inplace=True)


In [ ]:
news_df['text'] = news_df['title']+ ' ' + news_df['content']


# Preprocessing Second Step

In [9]:
def clean_text(text):
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text
news_df['clean_text'] = news_df['text'].apply(clean_text)

NameError: name 'news_df' is not defined

In [ ]:
stopwords = set(STOPWORDS)
totalTarget = ' '.join(news_df['clean_text'])
wordCloud = WordCloud(width=1000,height=500,stopwords=stopwords).generate(totalTarget)

In [ ]:
fig = px.imshow(wordCloud,color_continuous_scale='gray')
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

In [ ]:
totalTarget = totalTarget.split()
freq_dist = nltk.FreqDist(totalTarget)

In [ ]:
freq_dist = pd.DataFrame({'word':list(freq_dist.keys()),'count':list(freq_dist.values())})

In [ ]:
px.bar(freq_dist.nlargest(20,'count').sort_values('count'),y='word',x='count',
       title='Frequency of Words',color_discrete_sequence=['#137a63'])

# Prepare Data for Models

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(news_df.clean_text)
sequences = tokenizer.texts_to_sequences(news_df.clean_text)
text = pad_sequences(sequences)
labels = news_df.category_id.astype('float32')

In [ ]:
tokenizer.word_index['said']

In [ ]:
reverse_word_index = dict([(value,key) for (key,value) in tokenizer.word_index.items()])

In [ ]:
reverse_word_index[1]

In [ ]:
' '.join([reverse_word_index.get(i,'') for i in text[0]])

In [ ]:
news_df['clean_text'][0]


In [ ]:
def vectorize_sequence(sequnce,dimension=10000):
    results = np.zeros((len(sequnce),dimension))
    for i,sequnce in enumerate(sequnce):
        results[i,sequnce] = 1
    return results
vectorizer = vectorize_sequence(text)

In [ ]:
vectorizer[0]

In [ ]:
vec = TfidfVectorizer(max_features=10000)
features = vec.fit_transform(news_df.clean_text).toarray()
labels = news_df.category_id
print(type(features),features.shape)

In [ ]:
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(news_df,news_df['category_id']):
    strat_train_set = news_df.iloc[train_index]
    strat_test_set = news_df.iloc[test_index]

In [ ]:
X_train = strat_train_set['clean_text']
y_train = strat_train_set['category_id']

In [ ]:
X_test = strat_test_set['clean_text']
y_test = strat_test_set['category_id']

In [ ]:
one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [ ]:
Xtrain_tfidf = vec.fit_transform(X_train)
Xtest_tfidf = vec.fit_transform(X_test)


In [ ]:
Xtrain_vectorize = vectorize_sequence(text[train_index])
Xtest_vectorize = vectorize_sequence(text[test_index])

# Neural Network Model

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(5,activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

In [ ]:
history = model.fit(Xtrain_vectorize,one_hot_train_labels,
                   epochs=20,
                   batch_size=64,
                   validation_data=(Xtest_vectorize,one_hot_test_labels))

In [ ]:
model.predict(Xtest_vectorize)

In [ ]:
X_test[1713]

In [ ]:
y_test

In [ ]:
id_to_category

In [ ]:
history.history.keys()

In [ ]:
acc = history.history['acc']
acc_val = history.history['val_acc']
epochs = range(1,len(acc)+1)

In [ ]:
plt.plot(epochs,acc,'bo')
plt.plot(epochs,acc_val,'r')